In [75]:
import csv
import matplotlib.pyplot as plt
from functools import cmp_to_key

1. it has space for end moment small express packages
2. faster dues to direct implementation

In [76]:
class Item:
    def __init__(self, charid, length, width, height, weight, ptype, profit, itemid):
        #assuming that along x axis is width, along y axis is height and along z axis is depth/length
        self.itemid = int(itemid)
        self.width = int(width)
        self.height = int(height)
        self.length = int(length)
        self.profit = int(profit)
        self.weight = int(weight)
        self.charid = charid
        self.x = None
        self.y = None
        self.z = None
        self.ptype = ptype

    def __repr__(self):
        return f"Item({self.charid}, {self.length}, {self.width}, {self.height}, ({self.x}, {self.y}, {self.z}), {self.weight}, {self.profit}, {self.ptype})\n"


In [77]:
class Uld:
    #assuming that along x axis is width, along y axis is height and along z axis is depth/length
    def __init__(self, charid, length, width, height, capacity, uid):
        self.uid = int(uid)
        self.capacity = int(capacity)
        self.width = int(width)
        self.height = int(height)
        self.length = int(length)
        self.charid = charid
    def __repr__(self):
        return f"ULD({self.charid}, {self.length}, {self.width}, {self.height}, {self.capacity})\n"

In [78]:
#sort functions based on length, height and width
def sort_length(items, start=0, end=400, rev=False):
  '''sort in ascending using length'''
  unsorted_items = items[start:end]
  sorted_items = sorted(unsorted_items, key = lambda x: x.length, reverse = rev)
  return sorted_items

def sort_height(items, start = 0, end = 400,rev=False):
  '''sort in ascending using height'''
  unsorted_items = items[start:end]
  sorted_items = sorted(unsorted_items, key = lambda x: x.height, reverse = rev)
  return sorted_items

def sort_width(items, start = 0, end = 400, rev=True):
  '''sort in descending using width'''
  unsorted_items = items[start:end]
  sorted_items = sorted(unsorted_items, key = lambda x: x.width, reverse=rev)
  return sorted_items


In [79]:
#extract data from csv
uld_obj = open('ulds.csv', 'r')
item_obj = open('packages.csv', 'r')
data_item = csv.reader(item_obj)
ind = 0
priority_count = 0
total_delay_cost = 0

items=[]
ulds=[]

for row in data_item:
  if ind==0:
    ind=1
    continue
  row = [x.strip().replace("'", "").strip() for x in row]
  ptype = row[5]
  if(ptype=="Economy"):
      total_delay_cost += int(row[6])
      row.append(ind)
      item = Item(*row)
      #items_eco.append(item)
      items.append(item)
      #ind+=1

  else:
      priority_count +=1
      row[6] = 1000
      row.append(ind)
      item = Item(*row)
      #items_prio.append(item)
      items.append(item)

  ind+=1


data_uld = csv.reader(uld_obj)
ind = 0
for row in data_uld:
    if(ind==0):
        ind = 1
        continue
    row = [x.strip().replace("'", "").strip() for x in row]
    row.append(ind)
    container = Uld(*row)
    ulds.append(container)
    ind+=1

total_items = len(items)

In [80]:
'''
sorting in ascending order of height
then ascending order of height
then
'''
# sorting items based on increasing value of height , then length then decreasing order of width
items = sorted(items, key = lambda x : (x.height, x.length, -x.weight, -x.width))

# sorting ulds , initially smallest height, then length and greatest width
ulds = sorted(ulds, key = lambda x : (x.height, x.length, -x.width))
print(ulds)

[ULD(U1, 224, 318, 162, 2500)
, ULD(U2, 224, 318, 162, 2500)
, ULD(U3, 244, 318, 244, 2800)
, ULD(U4, 244, 318, 244, 2800)
, ULD(U5, 244, 318, 285, 3500)
, ULD(U6, 244, 318, 285, 3500)
]


In [104]:
def levelSlice(items, uld):
  placed_items = []
  i = 0
  retry =[0]*len(items)
  for uld in ulds:
    used_weight = 0
    used_width = 0 #used for filling across width
    used_height = 0 # initial level
    used_length = 0 # initial slice

    #uld param
    cap = uld.capacity
    binx = uld.length
    biny = uld.height
    binz = uld.width

    #item param
    max_item_length = 0
    max_item_height = 0

    while(i<len(items)):
      item = items[i]
      if(retry[i]==2):
        #trying at most 2 times to place an item
        i+=1
        continue
      if(item.weight + used_weight > cap):
        #checking weight constraints
        retry[i] += 1
        break
      elif(item.height + used_height > biny):
        # moves to next uld if item height exceeds allowed height
        retry[i] += 1
        break
      elif(item.width + used_width > binz):
        # moves to next slice if width of item is exceeded
        used_width = 0
        used_length = max_item_length
        max_item_length = 0
        retry[i] += 1
      elif(item.length + used_length > binx):
        used_length = 0
        used_width=0
        used_height = max_item_height
        max_item_height = 0
        retry[i] += 1
      else:
        max_item_length = max(max_item_length, item.length + used_length) # storing max length for next x slice
        max_item_height = max(max_item_height, item.height + used_height) # items already sorted in increasing order of height

        x = used_length # value of starting coordinate of slice in use
        y = used_height # value of the level at which the packages are placed
        z = used_width  # the width of the last item placed in that level-slice

        used_width += item.width #new starting width for next item

        used_weight += item.weight
        # assign coordinates to item
        item.x=x
        item.y=y
        item.z=z
        i+=1
        placed_items.append((uld.uid,item))
  print(retry)
  return placed_items





In [105]:
placed_items = levelSlice(items,ulds)
print(len(placed_items))

[0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 2, 1, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 2, 1, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [106]:
# writing to output
opfile = open("solution_level_slice.csv","w")
writer = csv.writer(opfile)
writer.writerow(["uld_id","pack_id","x1","y1","z1","x2","y2","z2"])
for row in placed_items:
  item = row[1]
  uid = row[0]
  charid = item.charid
  x1 = item.x
  y1 = item.y
  z1 = item.z
  x2 = x1 + item.length
  y2 = y1 + item.height
  z2 = z1 + item.width
  writer.writerow(["U"+str(uid), charid, x1,z1, y1, x2, z2, y2])
opfile.close()